In [2]:
# !pip3 install wikipedia-api

In [7]:
# ! pip install wikipedia

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=afcab6ee87dcb1a1f7d93a3bef71c072a8bbb8ba5cc6ce19dcb6cefcd9f58117
  Stored in directory: /home/codespace/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import openai
from key import OPENAI_API_KEY
import wikipediaapi
import wikipedia

In [3]:
movie100k = pd.read_csv('/workspaces/LLM-Recommender-System/data/processed_movie100k.csv')
movie100k.head()
print(movie100k.shape)

(100000, 7)


In [4]:
movie_data = []

for title in np.unique(movie100k['movie_title']):
    try:
        # Formatting the title for Wikipedia
        title_formatted = title.replace("'", "").rstrip()
        if "(" in title_formatted and ")" in title_formatted:
            title_formatted = title_formatted.replace(")", " film)")  # Add 'film' before the closing parenthesis
        else:
            title_formatted += " (film)"

        # Fetching the summary
        summary = wikipedia.summary(title_formatted, sentences=1)
        movie_data.append({'movie_title': title, 'wiki_summary': summary})
    except wikipedia.DisambiguationError as e:
        # Handle disambiguation by looking for a title containing 'film'
        relevant_page = next((option for option in e.options if "film" in option and title.split(" (")[0] in option), None)
        if relevant_page:
            try:
                summary = wikipedia.summary(relevant_page, sentences=1)
                movie_data.append({'movie_title': title, 'wiki_summary': summary})
            except Exception as ex:
                print(f"Failed to fetch summary for {title}: {ex}")
        else:
            print(f"No relevant Wikipedia page found for {title}")
    except Exception as ex:
        print(f"Failed to fetch summary for {title}: {ex}")

# Creating the DataFrame after collecting all data
movie_wiki = pd.DataFrame(movie_data)

Failed to fetch summary for 187 (1997): Page id "187 1994 film" does not match any pages. Try another id!
Failed to fetch summary for 2001: A Space Odyssey (1968): Page id "2001 a space odyssey 1998 film" does not match any pages. Try another id!
Failed to fetch summary for African Queen, The (1951): Page id "african queens the 1951 film" does not match any pages. Try another id!
Failed to fetch summary for Alice in Wonderland (1951): Page id "alice in wonderland 1981 film" does not match any pages. Try another id!
Failed to fetch summary for Amadeus (1984): Page id "amateur 1994 film" does not match any pages. Try another id!
Failed to fetch summary for Amityville 3-D (1983): Page id "amityville 3 d 1993 film" does not match any pages. Try another id!
Failed to fetch summary for Andre (1994): Page id "andres 1991 film" does not match any pages. Try another id!
Failed to fetch summary for April Fool's Day (1986): Page id "april fool's day 1986 film" does not match any pages. Try anothe

/home/codespace/.python/current/lib/python3.10/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/codespace/.python/current/lib/python3.10/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


No relevant Wikipedia page found for B*A*P*S (1997)
Failed to fetch summary for Babe (1995): Page id "babe 1999 film" does not match any pages. Try another id!
Failed to fetch summary for Babysitter, The (1995): Page id "babysitters the 1996 film" does not match any pages. Try another id!
Failed to fetch summary for Backbeat (1993): Page id "backchat 1995 film" does not match any pages. Try another id!
Failed to fetch summary for Bad Company (1995): Page id "bad company 1999 film" does not match any pages. Try another id!
Failed to fetch summary for Ballad of Narayama, The (Narayama Bushiko) (1958): Page id "ballad of narayama the narayama bushi kô film 1988 film" does not match any pages. Try another id!
Failed to fetch summary for Barcelona (1994): Page id "barcelona 1992 film" does not match any pages. Try another id!
Failed to fetch summary for Beautiful Girls (1996): Page id "beautiful girls 1998 film" does not match any pages. Try another id!
Failed to fetch summary for Beautiful

In [ ]:
movie_wiki

In [1]:
movie_wiki.to_csv('/workspaces/LLM-Recommender-System/data/movie_wiki.csv')

NameError: name 'movie_wiki' is not defined

In [15]:
# merge movie100k and movie_wiki
movie100k_with_wiki = movie100k.merge(movie_wiki, on='movie_title', how='left')
movie100k_with_wiki.head()

,user_id,movie_id,rating,timestamp,movie_title,genres,avg_rating,wiki_summary
0,1,1,5,874965758,Toy Story (1995),"Animation, Children",5.0,Toy Story is an American media franchise owned...
1,1,2,3,876893171,GoldenEye (1995),"Action, Adventure, Thriller",3.0,"GoldenEye is a 1995 spy film, the seventeenth ..."
2,1,3,4,878542960,Four Rooms (1995),Thriller,4.0,Four Rooms is a 1995 American anthology \nfarc...
3,1,4,3,876893119,Get Shorty (1995),Action,3.0,Get Shorty is a 1995 American gangster comedy ...
4,1,5,3,889751712,Copycat (1995),Thriller,3.0,Copycat is a 1995 American psychological thril...


In [37]:
print(f"Percentage of rows missing in the wiki_summary column: {round(movie100k_with_wiki['wiki_summary'].isna().sum() / movie100k_with_wiki.shape[0] * 100, 2)}%")

Percentage of rows missing in the wiki_summary column: 6.43%
